### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/5-N Gram In N Gram Analysis/Turkish"

# parameter
file = "Turkish_198_Twogram_In_6_Threegram_4_Fourgram_2_Fivegram_Sample_With_200_Word_Frequency_Result"  # Turkish Dutch SV
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "200_Threegram"
first_column = 2  # start column location number
end_column = 3
time_shift = 0.6
sample_num = 6

# youtube sentence condition
sent_time_start = True  # True, False
sent_time_start_time = 700 # sn

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [7]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [8]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [9]:
if sent_time_start:
    df_youtube_sentence = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_sentence.reset_index(drop=True, inplace=True)
else:
    df_youtube_sentence = df_youtube_sentence

df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
1050454,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
1050455,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
1050456,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
1050457,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [10]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram,frequency,threegram,three_freq,fourgram,four_freq,fivegram,five_freq
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,başka bir şey var,12619.0,daha önce böyle bir şey,2160.0
2,ben de,377765.0,başka bir şey,75193,bir şey daha var,12185.0,ne kadar güzel değil mi,1647.0
3,teşekkür ederim,370619.0,bir şey değil,68498,bir şey mi var,11834.0,güzel bir gün değil mi,1423.0
4,ne oldu,322758.0,bir şey mi,64933,de öyle değil mi,5379.0,sen de biliyorsun ben de,296.0
...,...,...,...,...,...,...,...,...
1119,NaN,NaN,işte yine yapıyorsun,12,NaN,NaN,NaN,NaN
1120,NaN,NaN,zaten tüm bu,32,NaN,NaN,NaN,NaN
1121,NaN,NaN,sen zaten tüm,14,NaN,NaN,NaN,NaN
1122,NaN,NaN,zaten tüm gün,14,NaN,NaN,NaN,NaN


In [11]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [12]:
len(search_list)

1124

In [13]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,57.900,59.160,bir şey yok abla,CSOIcsJSZe0
1,bir şey yok,95.583,97.694,merak edecek bir şey yok dedim ya,csLKMd7sazM
2,bir şey yok,498.340,505.680,özgürce bir şekilde yaşanabilmesini savunmak d...,h601hX4u56A
3,bir şey yok,643.730,649.040,öyle bir şey yok hani bu birinci evredir o zam...,2SS88Z9WIRw
4,bir şey yok,394.470,396.480,ama öyle bir şey yok aknenin yaşı yok,uiwgbWJfTyE
...,...,...,...,...,...
2592,aslında gerçek bir,645.070,650.890,karakteri aslında gerçek bir hayat hik yesinde...,QBbCk8B8KtY
2593,aslında gerçek bir,0.000,7.050,bu enquanto filminde bunu duydun mu bu gördüğü...,ACa3f16vhwY
2594,ama aslında gerçek,75.264,81.376,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
2595,şey aslında gerçek,191.020,200.880,yumurta düzeyi dediğimiz şey aslında gerçek an...,pQpngpFZjGk


In [14]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,57.900000,58.845000,bir şey yok abla,CSOIcsJSZe0
1,bir şey yok,96.350636,97.182242,merak edecek bir şey yok dedim ya,csLKMd7sazM
2,bir şey yok,504.579000,505.680000,özgürce bir şekilde yaşanabilmesini savunmak d...,h601hX4u56A
3,bir şey yok,643.976977,644.779651,öyle bir şey yok hani bu birinci evredir o zam...,2SS88Z9WIRw
4,bir şey yok,394.904595,395.610811,ama öyle bir şey yok aknenin yaşı yok,uiwgbWJfTyE
...,...,...,...,...,...
2592,aslında gerçek bir,645.621368,646.846632,karakteri aslında gerçek bir hayat hik yesinde...,QBbCk8B8KtY
2593,aslında gerçek bir,5.710500,7.050000,bu enquanto filminde bunu duydun mu bu gördüğü...,ACa3f16vhwY
2594,ama aslında gerçek,75.264000,76.876889,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
2595,şey aslında gerçek,193.537447,195.635319,yumurta düzeyi dediğimiz şey aslında gerçek an...,pQpngpFZjGk


In [15]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,57.300000,59.445000,bir şey yok abla,CSOIcsJSZe0
1,bir şey yok,95.750636,97.782242,merak edecek bir şey yok dedim ya,csLKMd7sazM
2,bir şey yok,503.979000,506.280000,özgürce bir şekilde yaşanabilmesini savunmak d...,h601hX4u56A
3,bir şey yok,643.376977,645.379651,öyle bir şey yok hani bu birinci evredir o zam...,2SS88Z9WIRw
4,bir şey yok,394.304595,396.210811,ama öyle bir şey yok aknenin yaşı yok,uiwgbWJfTyE
...,...,...,...,...,...
2592,aslında gerçek bir,645.021368,647.446632,karakteri aslında gerçek bir hayat hik yesinde...,QBbCk8B8KtY
2593,aslında gerçek bir,5.110500,7.650000,bu enquanto filminde bunu duydun mu bu gördüğü...,ACa3f16vhwY
2594,ama aslında gerçek,74.664000,77.476889,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
2595,şey aslında gerçek,192.937447,196.235319,yumurta düzeyi dediğimiz şey aslında gerçek an...,pQpngpFZjGk


In [16]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,57,59,bir şey yok abla,CSOIcsJSZe0
1,bir şey yok,96,98,merak edecek bir şey yok dedim ya,csLKMd7sazM
2,bir şey yok,504,506,özgürce bir şekilde yaşanabilmesini savunmak d...,h601hX4u56A
3,bir şey yok,643,645,öyle bir şey yok hani bu birinci evredir o zam...,2SS88Z9WIRw
4,bir şey yok,394,396,ama öyle bir şey yok aknenin yaşı yok,uiwgbWJfTyE
...,...,...,...,...,...
2592,aslında gerçek bir,645,647,karakteri aslında gerçek bir hayat hik yesinde...,QBbCk8B8KtY
2593,aslında gerçek bir,5,8,bu enquanto filminde bunu duydun mu bu gördüğü...,ACa3f16vhwY
2594,ama aslında gerçek,75,77,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI
2595,şey aslında gerçek,193,196,yumurta düzeyi dediğimiz şey aslında gerçek an...,pQpngpFZjGk


In [17]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bir şey yok,57,59,bir şey yok abla,CSOIcsJSZe0,https://www.youtube.com/watch?v=CSOIcsJSZe0&t=57s
1,bir şey yok,96,98,merak edecek bir şey yok dedim ya,csLKMd7sazM,https://www.youtube.com/watch?v=csLKMd7sazM&t=96s
2,bir şey yok,504,506,özgürce bir şekilde yaşanabilmesini savunmak d...,h601hX4u56A,https://www.youtube.com/watch?v=h601hX4u56A&t=...
3,bir şey yok,643,645,öyle bir şey yok hani bu birinci evredir o zam...,2SS88Z9WIRw,https://www.youtube.com/watch?v=2SS88Z9WIRw&t=...
4,bir şey yok,394,396,ama öyle bir şey yok aknenin yaşı yok,uiwgbWJfTyE,https://www.youtube.com/watch?v=uiwgbWJfTyE&t=...
...,...,...,...,...,...,...
2592,aslında gerçek bir,645,647,karakteri aslında gerçek bir hayat hik yesinde...,QBbCk8B8KtY,https://www.youtube.com/watch?v=QBbCk8B8KtY&t=...
2593,aslında gerçek bir,5,8,bu enquanto filminde bunu duydun mu bu gördüğü...,ACa3f16vhwY,https://www.youtube.com/watch?v=ACa3f16vhwY&t=5s
2594,ama aslında gerçek,75,77,ama aslında gerçek anlamda gerçek öyle değildi...,yXkiaO7_SKI,https://www.youtube.com/watch?v=yXkiaO7_SKI&t=75s
2595,şey aslında gerçek,193,196,yumurta düzeyi dediğimiz şey aslında gerçek an...,pQpngpFZjGk,https://www.youtube.com/watch?v=pQpngpFZjGk&t=...


In [18]:
df_word_group_time_loc_result["search_string"].nunique()

619

In [19]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [20]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Sheet1_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [ ]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass